In [13]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('../')

from src.models.models import *
from src.utils.modeling import *
from src.utils.preproc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### **Binary Classification Models Fine-Tuning**

**VGG 16**

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = preproc_pipeline(desired_magnification='200X', 
                                                    image_resolution=(224, 224), 
                                                    classification_type='binary')

In [15]:
vgg = binary_classification_vgg16_model(input_shape=(224, 224, 3))